# 11월 18일 실습2(동적 수집)(exam3.ipynb 관련)

In [7]:
#소스1
from selenium import webdriver
#from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time #time.sleep(숫자)
import re #regular expression. 정규표현식 사용시 필요한 API

import pandas as pd

goodsname = list()
goodsprice = list()

driver = webdriver.Chrome(service=Service('C:/Temp/chromedriver')) #셀레니움에 의해 제어되는 크롬브라우저 구동시키면서 웹브라우저 객체 생성
driver.implicitly_wait(3) #요청하고 응답을 받아올때 내부적으로 3초정도까지는 대기할 수 있다는 설정(한 번 설정하면 끝?)
driver.get('http://gs25.gsretail.com/gscvs/ko/products/event-goods') 
time.sleep(2) #사이트로 가고나서 렌더링할 시간을 줌.

byLinkText = driver.find_element(By.LINK_TEXT,'2+1 행사') #하이퍼링크 텍스트(컨텐트)가 정확히 '2+1 행사'인 것을 찾는다.
byLinkText.click() #하이퍼링크 클릭

for i in range(10) : #1이상 10미만(9번 반복)
    print(f"{i+1}페이지 스크래핑")
    names = driver.find_elements(By.CSS_SELECTOR, "#contents > div.cnt > div.cnt_section.mt50 > div > div > div:nth-child(5) > ul > li > div > p.tit") #상품명
    prices = driver.find_elements(By.CSS_SELECTOR, "#contents > div.cnt > div.cnt_section.mt50 > div > div > div:nth-child(5) > ul > li > div > p.price > span") #가격
    print(i+1,"페이지 제품 개수:",len(names))
    if len(names) == 0 :
        print("추출되는 상품이 더 이상 없다.")
        break #while True로 된 무한루프 종료
    #for name in names :
        #print(name.tag_name, ":", name.text)
    for j in range(len(names)) : #한 페이지에서 추출된 개수만큼 반복하여 화면에 내보낸다.
        print(re.sub("[,원]", '', prices[j].text)) #이건 exam1.ipynb 참고해야 한다.
        #prices[i]= re.sub("[,원]", '', prices[j].text) #헤맸던 흔적
        #prices[i] = prices[i].text.replace(',', '')    #헤맸던 흔적
        #prices[i] = prices[i].text.replace('원', '')   #헤맸던 흔적
        
        temp_price= re.sub("[,원]", '', prices[j].text) #기존 prices를 바꾸는 게 아니라 새로 임시 변수를 만듦.
        goodsname.append(names[j].text)
        goodsprice.append(temp_price)
                
        print(names[j].text, prices[j].text, sep=", ", end="\n") #한 페이지에 원래 50~60개씩 보여주는데 10개만 출력된다. 실제로 웹페이지에서 스크롤바를 내려보면 스크롤바의 크기가 줄어들면서 더 나온다. #스타벅스 홈페이지에서 3개만 있다가 스크롤바를 내리면 더 나왔던것과 같다. 즉 스크롤바 처리를 해줘야 한다.
    print("------------------------------------------------")
    #linkurl = '#app-root > div > div.XUrfU > div.zRM9F > a:nth-child(7)' #다음페이지로 옮겨가는 '>버튼'의 css선택자. #헤맸던 흔적
    try :
        #linkNum = driver.find_element(By.CSS_SELECTOR, linkurl) #헤맸던 흔적
        #nextBtn = driver.find_element(By.CSS_SELECTOR, "#contents > div.cnt > div.cnt_section.mt50 > div > div > div:nth-child(5) > div > a.next") #헤맸던 흔적
        nextBtn = driver.find_element(By.LINK_TEXT, "다음 페이지로 이동")
    except : #맨 끝페이지까지 가면 꺽새괄호버튼(다음페이지로 가는 버튼)이 사라지니까 무한반복문을 종료시킴
        print("더 이상 다음 페이지 없음")
        break
    nextBtn.click()  #클릭이벤트 발생시킨다.
    time.sleep(5) #클릭이벤트와 관련해서 시간을 기다려주는 것으로 기억한다. 
print("스크래핑 종료")
#driver.quit() #브라우저 창을 닫는다.


# csv 파일로 저장
goods_dict = {"상품명":goodsname, "가격":goodsprice} #딕셔너리
df = pd.DataFrame(goods_dict) #pd는 pandas #딕셔너리를 데이터프레임 형식으로 저장(pandas api)
display(df) #만든 데잉터프레임을 보여준다.
df.to_csv("output/gs25_twotoone.csv", index=False) #데이터프레임을 csv 파일로 저장한다.

1페이지 스크래핑
1 페이지 제품 개수: 8
1500
푸르밀)웰치사과에이드250ML, 1,500원
1500
푸르밀)웰치포도에이드250ML, 1,500원
15900
하겐)녹차파인트, 15,900원
15900
하겐)딸기파인트, 15,900원
15900
하겐)레드벨벳치즈케이크파인트, 15,900원
15900
하겐)마카다미아파인트, 15,900원
15900
하겐)바닐라라즈베리파인트, 15,900원
15900
하겐)바닐라밀크티파인트, 15,900원
------------------------------------------------
2페이지 스크래핑
2 페이지 제품 개수: 8
15900
하겐)바닐라파인트, 15,900원
15900
하겐)벨지안초콜릿&헤이즐넛파인, 15,900원
15900
하겐)쵸코렛파인트, 15,900원
15900
하겐)쿠키앤크림파인트, 15,900원
15900
하겐)트위스트초코딸기파인트, 15,900원
15900
하겐)트위스트초코바닐라파인트, 15,900원
1000
오뚜기)육개장(소컵), 1,000원
1500
그대로쫀득고구마스틱(20G), 1,500원
------------------------------------------------
3페이지 스크래핑
3 페이지 제품 개수: 8
3000
더스윗)틱톡젤리딸기3000, 3,000원
3000
더스윗)틱톡젤리포도3000, 3,000원
1800
피니)라즈앤드블랙베리85G, 1,800원
1800
피니)젤리빈90G, 1,800원
1800
피니)젤리웜즈100G, 1,800원
1800
피니)콜라보틀젤리100G, 1,800원
1800
피니)크로커다일젤리100G, 1,800원
1800
피니)피지웜즈100G, 1,800원
------------------------------------------------
4페이지 스크래핑
4 페이지 제품 개수: 8
1800
피니)해피믹스90G, 1,800원
1800
피니)해피사우워믹스90G, 1,800원
600
마즈)스니커즈미니20G, 600원
3900
거산)당근꿀맛고구마60G(봉

,상품명,가격
0,푸르밀)웰치사과에이드250ML,1500
1,푸르밀)웰치포도에이드250ML,1500
2,하겐)녹차파인트,15900
3,하겐)딸기파인트,15900
4,하겐)레드벨벳치즈케이크파인트,15900
...,...,...
75,팔도)비빔장스틱형(10입),7000
76,허쉬)머쉬멜로핫초코원컵30G,1800
77,허쉬)민트핫초코원컵30G,1800
78,티젠)핫콤부차(진저레몬)10입,7500
